In [238]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import numpy.ma as ma
from numpy import genfromtxt
from collections import defaultdict
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.layers import Dense
from tensorflow.keras.models import Sequential
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.model_selection import train_test_split
from sklearn.neighbors import NearestNeighbors
import tabulate
import random
import sympy
import scipy.linalg as sla
import matplotlib.pyplot as plt
# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [239]:
random.seed()
movies_data = pd.read_csv('/kaggle/input/top-movies/movies.csv')
users_data = pd.read_csv('/kaggle/input/mock-ratings/ratings.csv')
#print(movies_data)
#print(users_data)

In [240]:
#collaborative filtering start
users_data_copy = users_data.copy()
movies_data_copy = movies_data.copy()
users_data_copy = pd.get_dummies(users_data_copy) #one hot encoding to convert categorical data to numerical
movies_data_copy = pd.get_dummies(movies_data_copy) #one hot encoding to convert categorical data to numerical
#print(movies_data_copy)

users_data_copy = users_data_copy.astype('float64') 
users_data_copy = users_data_copy.to_numpy()  #converts to numpy
movies_data_copy = movies_data_copy.astype('float64') 
movies_data_copy = movies_data_copy.to_numpy()  #converts to numpy

#print(users_data_copy)
num_users = int(users_data_copy[105338,0])
#print("users_data_copy:", users_data_copy)
print("num_users is", num_users)

print("len(movies_data_copy[0]) is", len(movies_data_copy[0]))

num_movies = 10329
print("num_movies is", num_movies)

movieIDs = movies_data_copy[:,0]
usersDict = {}

for i in range(0, len(users_data_copy)): #for each user id, make a dictionary where key is movie id, and rating
    if(int(users_data_copy[i][0]) not in usersDict.keys()):
        usersDict[int(users_data_copy[i][0])] = {}
    usersDict[int(users_data_copy[i][0])][int(users_data_copy[i][1])] = int(users_data_copy[i][2]) 

for i in range(0, len(movieIDs)):
    movieIDs[i] = int(movieIDs[i])
#print(movieIDs)

#print(usersDict)


In [241]:
genres = movies_data["genres"]
#print(genres)
#genres = pd.get_dummies(genres)
#print(genres)


singular_genres = []
genresDict = {}
index = 0

for genre in genres:
    j = 0
    string = ""
    while(j < len(genre)):
        if(genre[j] == '|'):
            if(string not in genresDict.keys()):
                genresDict[string] = index
                index+=1
            string = "" 
        else:
            string += genre[j]
        j += 1
    if(string not in genresDict.keys()):
        genresDict[string] = index
        index+=1    
print(genresDict)

num_features = len(genresDict)

#X = np.zeros(num_features * num_movies)
#X = X.reshape(num_movies, num_features)
#print(X)

#for i in range(0, num_movies):
#    for j in range(0, num_features):
#        X[i][j] = random.uniform(-3, 3)
        
#print(X)

In [242]:
#Create table y, where each row is a movie, and each column is a user, and each [i,j] is a rating of movie i by user j
#Create table R to check if user rated a movie or not
y = []
R = []
#print(y)
for i in range(0, num_movies):
    y.append([])
    R.append([])
    for j in range(1, num_users+1):
        if(movieIDs[i] in usersDict[j].keys()):
            num = usersDict[j][movieIDs[i]]
            y[len(y)-1].append(usersDict[j][movieIDs[i]])
            if(num == 0):
                R[len(R)-1].append(0)
            else:
                R[len(R)-1].append(1)
        else:
            #y[len(y)-1].append(None)
            y[len(y)-1].append(0)
            R[len(R)-1].append(0)
        #print(y.shape)

In [243]:
#print(movies_data)
#print(genres)

for i in range(0, len(movieIDs)):
    genre = movies_data["genres"][i]
    #print(genre)
    j = 0
    string = ""
    idx = 0
    while(j < len(genre)):
        if(genre[j] == '|'):
            #if(string in genresDict.keys()):
            idx = genresDict[string]
            #X[i][idx] = 1
            string = ""
        else:
            string += genre[j]
        j+=1
    idx = genresDict[string]
    #X[i][idx] = 1
    string = ""

#print(genresDict)
#print(X)

In [245]:
print(num_users)
b = np.zeros(num_users * num_movies)
b = b.reshape(num_movies, num_users)
#print(b)
#b[:,:] = None 
#Making sample w, where w(subscript 1)(superscript j) = 5

W = np.zeros(num_features * num_users)
W = W.reshape(num_users, num_features)

for i in range(0, num_users):
    for j in range(0, num_features):
        W[i][j] = random.uniform(0, 5)

#print(W)

In [246]:
sols = []
X = np.array([])
#rows = 0
for i in range(0, num_movies):
    A = np.array([])
    B = np.array([])
    cols = 0
    for j in range(0, num_users):
        #divisor = y[i][j] / W[j][0]
        if(R[i][j] == 1):
            A = np.append(A, [W[j]])
            #A = np.append(A, [y[i][j]])
            B = np.append(B, [y[i][j]])
            cols+=1
    #print("A.shape is", A.shape)
    #print("B.shape is", B.shape)
    #print("B is", B)
    A = A.reshape(cols, num_features)
    arr = np.linalg.lstsq(A,B)
    #print(arr)
    #X = np.append(X, np.linalg.lstsq(A, B))
    X = np.append(X, arr[0])
    #print("A.shape after reshaping is", A.shape)
    #break

#A = sympy.Matrix(A).rref()

#X = np.append(X, np.linalg.lstsq(A,B))
#print("np.linalg.lstsq(A,B) is\n", np.linalg.lstsq(A,B))
print("X.shape is", X.shape)
#print("X is\n",X)
X = X.reshape(num_movies, num_features)
print("X.shape after is", X.shape)
#print("X after is\n",X)
#print("W[0] is \n", W[0])
#print("X[0] is \n", X[0])

plt.xlim(0, 10)
plt.ylim(0, 10)
plt.grid()

for i in range(0, 10):
    for j in range(0, num_users):
        if(R[i][j] == 1):
            _ = plt.plot(i, np.dot(X[i], W[j]), 'o', markersize = 5, markerfacecolor = "blue")
            _ = plt.plot(i, y[i][j], 'o', markersize = 5, markerfacecolor = "red")

_ = plt.legend()
plt.show()
#print(np.dot(X[0], W[0]))


In [251]:
def cost_function(W, b, X, R, y, num_users, num_movies, num_features, lambda_):
    print("In cost function")
    summ = 0
    for i in range(0, num_movies):
        for j in range(0, num_users):
            #print("i is {} and j is {}".format(i, j))
            if R[i][j] == 1:
                summ += (np.dot(W[j], X[i]) + b[0][j] - y[i][j])**2
    summ /= 2
    
    summ2 = 0
    for j in range(0, num_users):
        for k in range(0, num_features):
            summ2 += (W[j][k] ** 2)
    summ2 *= (lambda_/2)
    
    summ3 = 0
    for i in range(0, num_movies):
        for k in range(0, num_features):
            summ3 += (X[i][k] ** 2)
    summ3 *= (lambda_/2)
    
    #print("Final summ is", summ+summ2+summ3)
    return summ + summ2 + summ3

def cost_function_v2(X, W, b, y, R, lambda_):
    j = (tf.linalg.matmul(X, tf.transpose(W)) + b - y)*R
    J = 0.5 * tf.reduce_sum(j**2) + (lambda_/2) * (tf.reduce_sum(X**2) + tf.reduce_sum(W**2))
    return J

In [252]:
# Reduce the data set size so that this runs faster
num_users_r = 4
num_movies_r = 5 
num_features_r = 3
y_r = []
R_r = []

X_r = X[:num_movies_r, :num_features_r]
W_r = W[:num_users_r,  :num_features_r]
b_r = b[0, :num_users_r].reshape(1,-1)
#print(b_r)
for i in range(0, num_movies_r):
    y_r.append([])
    for j in range(0, num_users_r):
        y_r[len(y_r)-1].append(y[i][j])
#y_r = y[:num_movies_r, :num_users_r]
for i in range(0, num_movies_r):
    R_r.append([])
    for j in range(0, num_users_r):
        R_r[len(R_r)-1].append(R[i][j])

print(R_r)
#R_r = R[:num_movies_r, :num_users_r]

# Evaluate cost function
#J = cost_function(X_r, W_r, b_r, y_r, R_r, 0);

J = cost_function(W_r, b_r, X_r, R_r, y_r, num_users_r, num_movies_r, num_features_r, 0)
print(f"Cost: {J:0.2f}")

In [253]:
J = cost_function(W_r, b_r, X_r, R_r, y_r, num_users_r, num_movies_r, num_features_r, 1.5)
print(f"Cost: {J:0.2f}")

In [270]:
movielist = []
for i in movies_data['title']:
    movielist.append(i)
#print(movielist)

averages = []
number_of_ratings = []
for i in range(0, num_movies):
    avg = 0
    num = 0
    for j in range(0, num_users):
        if(R[i][j] == 1):
            avg += y[i][j]
            num+=1
    if(num != 0):
        avg /= num
        #averages.append(round(avg,2))
        averages.append(avg)
    else:
        averages.append(None)
    number_of_ratings.append(num)

data = []
for i in range(0, num_movies):
    data.append([averages[i], number_of_ratings[i], movielist[i]])

#created a dataframe
df = pd.DataFrame(data, columns=['mean rating', 'number of ratings', 'title'])
  
# print dataframe.
df

In [271]:
my_ratings = [0] * num_movies
my_ratings[929]  = 5   # Lord of the Rings: The Return of the King, The
my_ratings[246]  = 5   # Shrek (2001)
my_ratings[2716] = 3   # Inception
my_ratings[1150] = 5   # Incredibles, The (2004)
my_ratings[382]  = 2   # Amelie (Fabuleux destin d'Amélie Poulain, Le)
my_ratings[366]  = 5   # Harry Potter and the Sorcerer's Stone (a.k.a. Harry Potter and the Philosopher's Stone) (2001)
my_ratings[622]  = 5   # Harry Potter and the Chamber of Secrets (2002)
my_ratings[988]  = 3   # Eternal Sunshine of the Spotless Mind (2004)
my_ratings[2925] = 1   # Louis Theroux: Law & Disorder (2008)
my_ratings[2937] = 1   # Nothing to Declare (Rien à déclarer)
my_ratings[793]  = 5   # Pirates of the Caribbean: The Curse of the Black Pearl (2003)
my_rated = [i for i in range(len(my_ratings)) if my_ratings[i] > 0]
print(my_rated)
#print("\n My ratings:\n", my_ratings)
print("\nNew user ratings:\n")
for i in range(len(my_ratings)):
    if my_ratings[i] > 0 :
        print(f'Rated {my_ratings[i]} for  {df.loc[i,"title"]}');

In [285]:
new_y = []
new_R = []
for i in range(0, num_movies):
    new_y.append([])
    new_R.append([])
    for j in range(0, num_users-1):
        num = random.uniform(0, 5)
        guess = num - int(num)
        #if(guess >= 0.5):
        #    new_y[len(new_y)-1].append(int(num)+1)
        #else:
        #    new_y[len(new_y)-1].append(int(num))
        #if(new_y[i][j] == 0):
        #    new_R[len(new_R)-1].append(0)
        if(guess >= 0.3):
            new_y[len(new_y)-1].append(num)
        else:
            new_y[len(new_y)-1].append(0.0)
        if(new_y[i][j] == 0.0):
            new_R[len(new_R)-1].append(0)
        else:
            new_R[len(new_R)-1].append(1)

new_y = np.c_[my_ratings, new_y]

#for i in range(len(my_ratings)):
#new_R = np.c_[my_ratings, new_R]
for i in range(0, num_movies):
    num = int(bool(my_ratings[i]))
    new_R[i].insert(0, num)

#print("new_y[793] is\n", new_y[793])
#print("new_R[793] is\n", new_R[793])
print("new_y[0] is\n", new_y[0])
print("new_R[0] is\n", new_R[0])

In [286]:
def means(y, R):
    averages = []
    for i in range(0, len(y)):
        avg = 0
        num = 0
        for j in range(0, len(y[i])):
            if(R[i][j] > 0):
                avg += y[i][j]
                num += 1
        avg /= num
        averages.append(round(avg,2))
    return averages

In [287]:
def normalization(y, R):
    averages = means(y, R)
    y_norm = []
    for i in range(0, len(y)):
        y_norm.append([])
        for j in range(0, len(y[i])):
            if (y[i][j] > 0):
                y_norm[len(y_norm)-1].append(round(y[i][j] - averages[i],2))
            else:
                y_norm[len(y_norm)-1].append(0)
    return y_norm, averages

In [288]:
new_y_norm, new_y_means = normalization(new_y, new_R)

#print(len(new_y_norm), len(new_y_norm[0]))
#print(len(new_y_means), len(new_y_means[0]))

In [289]:
##Using tensorflow to get best values for parameters for minimum loss (gradient descent)

num_movies, num_users = len(y), len(y[0])
num_features = 20
print("num_movies is", num_movies)
print("num_users is", num_users)
print("num_features is", num_features)

tf.random.set_seed(1234) #using tf.Variable to track variables
W = tf.Variable(tf.random.normal((num_users, num_features), dtype=tf.float64), name='W')
X = tf.Variable(tf.random.normal((num_movies, num_features), dtype=tf.float64), name='X')
b = tf.Variable(tf.random.normal((1, num_users), dtype=tf.float64), name='b')

print("W.shape is", W.shape)
print("X.shape is", X.shape)
print("b.shape is", b.shape)

optimizer = keras.optimizers.Adam(learning_rate=1e-1)

In [290]:
#Need to repeat the following until convergence:
    #Compute forward pass
    #Compute the derivatives of the loss relative to parameters
    #Update parameters using the learning rate and the computer derivatives
    
    
iterations = 200
lambda_ = 1
#print("Here")
for i in range(0, iterations+1):
    #print("i is", i)
    with tf.GradientTape() as tape:
        #cost_value = cost_function(X, W, b, Yn)
        #cost_value = cost_function(W, b, X, new_R, new_y_norm, num_users, num_movies, num_features, lambda_)
        #print("cost_value is", cost_value)
        cost_value = cost_function_v2(X, W, b, new_y_norm, new_R, lambda_)
    grads = tape.gradient(cost_value, [X,W,b]) #derivative of cost function with respect to X, W, and b
    optimizer.apply_gradients(zip(grads, [X,W,b]))
    
    if(i%20 == 0):
        print(f"Training loss at iteration {i}: {cost_value:0.1f}")

In [291]:
#print(new_y_means)
new_y_means = np.array(new_y_means)
new_y_means = new_y_means.reshape(num_movies, 1)
print(new_y_means.shape)

In [292]:
# Make a prediction using trained weights and biases
p = np.matmul(X.numpy(), np.transpose(W.numpy())) + b.numpy()
print("p is\n",p)
print("new_y_means is\n", new_y_means)

#restore the mean
pm = p + new_y_means

print("pm is\n", pm)

my_predictions = pm[:,0]

# sort predictions
ix = tf.argsort(my_predictions, direction='DESCENDING')

for i in range(17):
    j = ix[i]
    if j not in my_rated:
        print(f'Predicting rating {my_predictions[j]:0.2f} for movie {movielist[j]}')

print('\n\nOriginal vs Predicted ratings:\n')
for i in range(len(my_ratings)):
    if my_ratings[i] > 0:
        print(f'Original {my_ratings[i]}, Predicted {my_predictions[i]:0.2f} for {movielist[i]}')


In [293]:
filter=(df["number of ratings"] > 20)
df["pred"] = my_predictions
df = df.reindex(columns=["pred", "mean rating", "number of ratings", "title"])
df.loc[ix[:300]].loc[filter].sort_values("mean rating", ascending=False)